In [2]:
import os

from tensorflow import keras
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
SMALLER_WIDTH = 600 // 3
SMALLER_HEIGHT = 450 // 3


def get_model() -> keras.Model:
    base_model = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=(SMALLER_HEIGHT, SMALLER_WIDTH, 3))

    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = keras.layers.Dropout(.2)(x)
    x = keras.layers.GlobalAveragePooling2D()(x)
    predictions = keras.layers.Dense(7, activation='softmax')(x)
    model = keras.Model(inputs=base_model.input, outputs=predictions)

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
train_dir = os.path.join('..', 'data', 'images_original_inception_resnet_v2_200x150_categorized', 'training')
valid_dir = os.path.join('..', 'data', 'images_original_inception_resnet_v2_200x150_categorized', 'validation')
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 200),
    batch_size=32,
    class_mode='categorical')
valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(150, 200),
    batch_size=32,
    class_mode='categorical')